# Content-based Filtering

### Loading Libraries

In [28]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import tag_mapping_sorted 
import warnings
warnings.filterwarnings('ignore')

Loading Datasets

In [29]:
genome_tags = pd.read_csv('../../data/ml-25m/genome-tags.csv')
links = pd.read_csv('../../data/ml-25m/links.csv')
movies = pd.read_csv('../../data/ml-25m/movies.csv')

### Tags aufbereiten

In [30]:
# Ersetze Tags in genome-tags.csv
# genome_tags['tag'] = genome_tags['tag'].replace(tag_mapping_sorted.tag_mapping)

# # Kombiniere genome_scores und genome_tags
# genome = pd.merge(genome_scores, genome_tags, on='tagId')

# # Ersetze Tags gemäß Mapping
# genome['tag'] = genome['tag'].replace(tag_mapping_sorted.tag_mapping)

# # Aggregiere Relevanzwerte (Durchschnitt)
# genome_agg = genome.groupby(['movieId', 'tag']).agg({'relevance': 'mean'}).reset_index()

# # Pivot-Tabelle neu erstellen
# genome_pivot = genome_agg.pivot_table(index='movieId', columns='tag', values='relevance', fill_value=0)

# Gib die ersten Zeilen der aggregierten Pivot-Tabelle aus (zur Überprüfung)
# print(genome_pivot.head(5))

### Korrelationen in Tags

In [31]:
# # Anzahl der einzigartigen Tags nach dem Mapping
# unique_tags_count = len(genome_pivot.columns)
# print(f"Anzahl der einzigartigen Tags nach dem Mapping: {unique_tags_count}")


# corr_matrix_tags = genome_pivot.corr()
# # print(corr_matrix_tags)

# # Unstack die Korrelationsmatrix und sortiere die Werte
# corr_pairs = corr_matrix_tags.unstack()
# sorted_pairs = corr_pairs.sort_values(kind="quicksort", ascending=False)

# # Entferne die 1.0 Korrelationswerte (Selbstkorrelationen)
# strong_pairs = sorted_pairs[sorted_pairs != 1]

# # Ausgabe der stärksten Korrelationen (z.B. Top 10)
# high_corr_pairs = strong_pairs[strong_pairs > 0.8] 
# print(high_corr_pairs)
# print(f"Anzahl der Paare mit hoher Korrelation: {len(high_corr_pairs)}")

# Speichern der hohen Korrelationen in einer separaten Datei
# with open('high_correlation_pairs.py', 'w') as f:
#     f.write("high_corr_pairs = {\n")
#     for (tag1, tag2), correlation in high_corr_pairs.items():
#         f.write(f"    '{tag1}': '{tag2}',\n")
#     f.write("}\n")

### Genres aufbereiten


In [32]:
# Entferne "(no genres listed)" aus der Genre-Liste
movies['genres'] = movies['genres'].replace('(no genres listed)', '')

# Trenne die Genres in separate Listen
genre_list = movies['genres'].str.split('|')

# Finde alle einzigartigen Genres
all_genres = set(genre for sublist in genre_list for genre in sublist if genre)

# Erstelle für jedes Genre eine Spalte und fülle sie mit binären Werten
for genre in all_genres:
    movies[genre] = movies['genres'].apply(lambda x: int(genre in x.split('|')))

# Entferne die ursprüngliche 'genres' Spalte
movies = movies.drop(columns=['genres'])

KeyError: 'genres'

### Titel bereinigen

In [ ]:
# Extrahiere das Erscheinungsjahr aus dem Titel und füge es als neue Spalte hinzu
movies['year'] = movies['title'].str.extract(r'\((\d{4})\)')

# Entferne das Erscheinungsjahr aus dem Titel
movies['title'] = movies['title'].str.replace(r'\s*\(\d{4}\)', '', regex=True)

### Merkmale zusammenführen

In [ ]:
# Merge der Tag-Relevanzen mit dem Movies DataFrame
# movies_with_tags = pd.merge(movies, genome_pivot, on='movieId', how='left').fillna(0)
# print(movies_with_tags.head(10))

### Nach Null-Werten suchen

In [ ]:
movies.info()
movies.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62423 entries, 0 to 62422
Data columns (total 22 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   movieId      62423 non-null  int64 
 1   title        62423 non-null  object
 2   Documentary  62423 non-null  int64 
 3   IMAX         62423 non-null  int64 
 4   Mystery      62423 non-null  int64 
 5   War          62423 non-null  int64 
 6   Action       62423 non-null  int64 
 7   Animation    62423 non-null  int64 
 8   Horror       62423 non-null  int64 
 9   Thriller     62423 non-null  int64 
 10  Adventure    62423 non-null  int64 
 11  Drama        62423 non-null  int64 
 12  Crime        62423 non-null  int64 
 13  Comedy       62423 non-null  int64 
 14  Fantasy      62423 non-null  int64 
 15  Western      62423 non-null  int64 
 16  Children     62423 non-null  int64 
 17  Romance      62423 non-null  int64 
 18  Musical      62423 non-null  int64 
 19  Sci-Fi       62423 non-nu

movieId          0
title            0
Documentary      0
IMAX             0
Mystery          0
War              0
Action           0
Animation        0
Horror           0
Thriller         0
Adventure        0
Drama            0
Crime            0
Comedy           0
Fantasy          0
Western          0
Children         0
Romance          0
Musical          0
Sci-Fi           0
Film-Noir        0
year           410
dtype: int64

In [ ]:
# numeric_cols = movies.select_dtypes(include=[float, int]).columns
# corr_matrix = movies[numeric_cols].corr()

# plt.figure(figsize=(12, 10))
# sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap='coolwarm')
# plt.show()

# print(movies[movies['title'].str.contains("Toy Story", case=False, na=False)])

Movies without year

In [ ]:
# Filme ohne Erscheinungsjahr ausgeben
movies_without_year = movies[movies['year'].isna()]
# print(movies_without_year)
# print(movies['year'].isna().sum())

movies_without_year.to_csv('movies_without_year.csv', index=False)
print("Movies without year saved to 'movies_without_year.csv'")

# Entferne Filme ohne Erscheinungsjahr
# movies = movies.dropna(subset=['year'])
# movies.isnull().sum()

Movies without year saved to 'movies_without_year.csv'


### Feature Set 1: Genre

In [ ]:
# Berechnung der Cosine Similarity basierend auf den Genres
genre_columns = list(all_genres)  
# genre_matrix = movies_with_tags[genre_columns].values
cosine_sim = cosine_similarity(genre_matrix, genre_matrix)

# Funktion zur Empfehlung von Filmen basierend auf Genres
def recommend_movies_by_genre(movie_title, cosine_sim=cosine_sim):
    # Index des eingegebenen Films finden
    idx = movies_with_tags[movies_with_tags['title'] == movie_title].index[0]
    
    # Ähnlichkeitsscores des Films mit allen anderen Filmen
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    # Sortierung der Filme basierend auf der Ähnlichkeit
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Indizes der 10 ähnlichsten Filme erhalten
    sim_scores = sim_scores[1:11]
    
    # Filmtitel der 10 ähnlichsten Filme ausgeben
    movie_indices = [i[0] for i in sim_scores]
    return movies_with_tags['title'].iloc[movie_indices]



NameError: name 'movies_with_tags' is not defined

### Test

In [ ]:
recommended_movies = recommend_movies_by_genre('Toy Story')
print(recommended_movies)